### Exp3.1_L_LeedsDataset_Onlytasks_CompvsIncomp_Clustering_supervised_layers
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Studying the Action feature vectors

In [1]:
import os
from six.moves import xrange 
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io
from sklearn import metrics
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')


# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112

cluster_length=16
feature_size=8192
nb_classes=2
saved_path='/nobackup/leopauly/S2l/'

batch_size=32
memory_batch_size_train=266
memory_batch_size_test=170
next_batch_start=0
sample_batch_size=10

In [3]:
print('Available GPU details:')
!nvidia-smi

Available GPU details:
Thu Jan 23 18:22:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:03:00.0 Off |                    0 |
| N/A   28C    P0    24W / 250W |      0MiB / 12198MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   29C    P0    25W / 250W |      0MiB / 12198MiB |      0%      Default |
+-------------------------------+----------------------+---------

####  Loading model 

In [4]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_ucf101_training_model_tf(summary=False)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

Miscellenious items finished..!!


####  Feature extraction

In [5]:
## Getting 16 frames after unifrom sampling of video sample
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):
    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr) 
  #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [6]:
layer_name='pool3/MaxPool3D:0'
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name(layer_name) 
    #('flatten_1/Reshape:0') #('dropout_2/cond/Merge:0') #('fc8/Relu:0')
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

In [7]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model 
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/activity_model.ckpt-67
Model restored from file: /nobackup/leopauly/S2l/


In [8]:
## Feature extraction
def get_features_from_class(class_folder):
    feature_set_a=[]
    base_dir_a=class_folder
    sub_dir_a=os.listdir(base_dir_a)
    sub_dir_a=sorted(sub_dir_a)
    if '.DS_Store' in sub_dir_a:
        sub_dir_a.remove('.DS_Store')
    print('\nSub directories:\n',sub_dir_a)
    for sub_dir_a_ in sub_dir_a:
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        feature_set_a.append(extract_video_features(vid_a))
    return np.array(feature_set_a)

#### Feature extraction

In [9]:
feature_set_a=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Push/')


Sub directories:
 ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']


In [10]:
feature_set_b=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/')


Sub directories:
 ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']


In [11]:
feature_set_c=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach n Push/')


Sub directories:
 ['Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']


#### Feature selection

In [12]:
## selecting features based on the indexes
def sel_feat(org_features,select_feature_cols):
    selected_features = [org_features[int(i)] for i in select_feature_cols]
    return np.array(selected_features)

def select_features(feature_set,select_feature_cols):
    improve_features=[]
    for i in range(0,len(feature_set)):
        improve_features.append(sel_feat(feature_set[i],select_feature_cols))
    improve_features=np.array(improve_features)
    print('Feature selected features:',improve_features.shape)
    return improve_features

#### Clustering 

In [13]:
## Clustering based on selected percentage of features
percentage_of_features=np.array([.2,.4,.6,.8,1])
for percent in (percentage_of_features):
    file_name='sel_feat_idx_'+layer_name.translate({ord('/'): None})+'_'+str(percent)+'.txt'
    select_feature_cols=np.loadtxt(file_name)
    print('Selected feature indexes loadded from file',file_name,'\n')
    
    feature_set_a_imporv=select_features(feature_set_a,select_feature_cols)
    feature_set_b_imporv=select_features(feature_set_b,select_feature_cols)
    feature_set_c_imporv=select_features(feature_set_c,select_feature_cols)

    kmeans = KMeans(n_clusters=3,random_state=1)
    points=np.concatenate((feature_set_a_imporv,feature_set_b_imporv,feature_set_c_imporv),axis=0)
    y_km = kmeans.fit_predict(points)

    ## Clustering evaluation metrics
    y_true=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
    print('\nAdjusted random score:',round(metrics.adjusted_rand_score(y_true,y_km),2))
    print('Adjusted mutual infromation score:',round(metrics.adjusted_mutual_info_score(y_true,y_km),2))
    print('Homogeneity score:',round(metrics.homogeneity_score(y_true,y_km),2),'\n')

Selected feature indexes loadded from file sel_feat_idx_pool3MaxPool3D:0_0.2.txt 

Feature selected features: (14, 40140)
Feature selected features: (14, 40140)
Feature selected features: (14, 40140)

Adjusted random score: 0.33
Adjusted mutual infromation score: 0.28
Homogeneity score: 0.31 

Selected feature indexes loadded from file sel_feat_idx_pool3MaxPool3D:0_0.4.txt 

Feature selected features: (14, 80281)
Feature selected features: (14, 80281)
Feature selected features: (14, 80281)

Adjusted random score: 0.33
Adjusted mutual infromation score: 0.41
Homogeneity score: 0.44 

Selected feature indexes loadded from file sel_feat_idx_pool3MaxPool3D:0_0.6.txt 

Feature selected features: (14, 120422)
Feature selected features: (14, 120422)
Feature selected features: (14, 120422)

Adjusted random score: 0.28
Adjusted mutual infromation score: 0.25
Homogeneity score: 0.28 

Selected feature indexes loadded from file sel_feat_idx_pool3MaxPool3D:0_0.8.txt 

Feature selected features: (1